In [44]:
import pandas as pd
import numpy as np
import sys
import re

In [24]:
# pip install soynlp

In [16]:
# pdf 추출 모듈 import

%run module_pdf_extractor.ipynb

In [17]:
# 이미 추출되어 있는 공시데이터 텍스트 가져오기

path = './1-2. ESG report/'
file_list = os.listdir(path+'output/')
file_list = [file for file in file_list if file.startswith("ESG_text")]

In [19]:
# 각 공시 보고서별 e/s/g 페이지 번호 미리 저장
# e/s/g 순서

dict = {
    'GSretail_2023':[range(7,18), range(18,34),range(2,7)],
    'hyundaehomeshopping_2023' : [range(6,16), range(16,32), range(2,6)],
    'LGlife_2023' : [range(5,22), range(22,53), range(2,5)],
    'lottehi_2023' : [range(5,14), range(14,32), range(2,5)],
    'sinsegye_2023' : [range(6,18), range(18,35), range(2,6)],
}

e/s/g별 텍스트 모두 모아서 각각 텍스트 파일로 저장

In [6]:
# e/s/g별 토큰 리스트 만들기
# e = []
# s = []
# g = []

In [20]:
e = ''
s = ''
g = ''

In [21]:
for file in file_list:
    name = file.replace('ESG_text_','').replace('.csv','')
    data = json_to_csv(path, '', name)

    for j in range(len(data["elements"])):
        if 'Text' in data["elements"][j]:
            page = data["elements"][j]['Page']
            if page in dict[name][0]:
                    e = e + data["elements"][j]['Text'] + ' '
            elif page in dict[name][1]:
                    s = s + data["elements"][j]['Text'] + ' '
            elif page in dict[name][2]:
                    g = g + data["elements"][j]['Text'] + ' '
            else:
                pass
        else:
            pass

In [37]:
file = open('e_doublespacelinecorpus.txt', 'w', encoding = 'utf-8')
file.write(e+' ')
file.close()
file = open('s_doublespacelinecorpus.txt', 'w', encoding = 'utf-8')
file.write(s+' ')
file.close()
file = open('g_doublespacelinecorpus.txt', 'w', encoding = 'utf-8')
file.write(g+' ')
file.close()

텍스트 파일로 저장된 e/s/g별 문서 불러와서 명사 추출

Soynlp의 LRNounExtractor_v2 이용

In [63]:
paths = ['e_doublespacelinecorpus.txt', 's_doublespacelinecorpus.txt', 'g_doublespacelinecorpus.txt']
nouns = []

In [64]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

for path in paths:
    corpus_path = path
    sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

    noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
    noun_extractor.train(sents)
    nouns.append(noun_extractor.extract())

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 5152 from 2653 sents. mem=0.150 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=11571, mem=0.150 Gb
[Noun Extractor] batch prediction was completed for 1370 words
[Noun Extractor] checked compounds. discovered 95 compounds
[Noun Extractor] postprocessing detaching_features : 667 -> 667
[Noun Extractor] postprocessing ignore_features : 667 -> 660
[Noun Extractor] postprocessing ignore_NJ : 660 -> 660
[Noun Extractor] 660 nouns (95 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.151 Gb                    
[Noun Extractor] 53.06 % eojeols are covered
[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 9207 from 5093 sents. 

In [72]:
# 빈도수만큼 해당 명사를 리스트에 저장

for i, n in enumerate(nouns):
    globals()[f"keys_{i}"] = []
    for w, v in n.items():
        for j in range(int(v[0])):
            locals()[f"keys_{i}"].append(w)
    globals()[f"keys_{i}"] = [k for k in locals()[f"keys_{i}"] if k.isalpha()]
    globals()[f"keys_{i}"] = [k for k in locals()[f"keys_{i}"] if not re.match(r"\d{4}년", k)]

In [7]:
# 단순 띄어쓰기 단위 토큰 추출

# for file in file_list:
#     name = file.replace('ESG_text_','').replace('.csv','')
#     data = json_to_csv(path, '', name)

#     for j in range(len(data["elements"])):
#         if 'Text' in data["elements"][j]:
#             page = data["elements"][j]['Page']
#             if page in dict[name][0]:
#                     e = e + (data["elements"][j]['Text']).split(' ')
#             elif page in dict[name][1]:
#                     s = s + (data["elements"][j]['Text']).split(' ')
#             elif page in dict[name][2]:
#                     g = g + (data["elements"][j]['Text']).split(' ')
#             else:
#                 pass
#         else:
#             pass

In [80]:
e = [i for i in keys_0 if len(i)>2 and not i.isdigit()]
s = [i for i in keys_1 if len(i)>2 and not i.isdigit()]
g = [i for i in keys_2 if len(i)>2 and not i.isdigit()]

In [81]:
# tf-idf 계산 전 전체 명사 수
print(len(e), len(s), len(g))

1679 4029 991


TF-IDF 계산

In [82]:
docs = [e, s, g]

In [76]:
def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return np.log(len(docs)/(df+sys.float_info.min))

def tfidf(t, d):
  return tf(t,d)* idf(t)

In [83]:
e_corpus = pd.DataFrame(e,columns= ['word'])
s_corpus = pd.DataFrame(s,columns= ['word'])
g_corpus = pd.DataFrame(g,columns= ['word'])

In [113]:
# 계산 및 내림차순으로 정렬
corpus = [(e_corpus,e), (s_corpus,s), (g_corpus, g)]

for c in corpus:
    for i, item in c[0].iterrows():
        c[0].loc[i,'tf-idf'] = tfidf(item['word'],c[1])
    c[0].drop_duplicates(inplace = True)
    c[0].sort_values(by = 'tf-idf', ascending=False, inplace=True)
    c[0].reset_index(inplace=True, drop = True)

In [114]:
for c in corpus:
    print(len(c[0]))

255
910
160


TF-IDF 점수가 1보다 큰 단어만 코퍼스로 구축

In [115]:
esg = ('e','s','g')
corp = (e_corpus, s_corpus, g_corpus)
corpus = pd.DataFrame(columns=['word', 'label', 'tf-idf'])

for i, c in zip(esg, corp):
    corpus = pd.concat([corpus,c[c['tf-idf']>1]])
    corpus.reset_index(drop=True)
    corpus.loc[len(corpus)-1,'label'] = i

corpus.fillna(method = 'bfill', inplace=True)
corpus.dropna(inplace=True)

In [119]:
# 기업명 관련 키워드 제거

corpus = corpus[~corpus['word'].str.contains(r"현대홈쇼핑|GS|롯데|신세계|LG", regex=True)]
corpus.reset_index(drop = True, inplace= True)

In [121]:
path = './1-2. ESG report/'
if not os.path.isdir(path + 'output/'):
    os.mkdir(path + 'output/')
corpus.to_csv(path + 'output/'+'corpus_esg_keyword_v2'+'.csv', index=False, encoding='utf-8-sig')